In [1]:
# IMPORTS
import training
import OBC.networks
import data_loader as dl
import Piggyback.networks as pbnet
import Rebuffi.networks as rbnet
import torch
import torch.optim as optim
import torch.nn as nn
import torch.utils.data
import copy

from OBC.ClassificationMetric import ClassificationMetric
from Piggyback.networks import *
from Piggyback.custom_layers import *
from Rebuffi.networks import *

In [2]:
folder = '/home/fcdl/Develop/Data/sample'
batch = 4
rgb = True
depth = False

from Depth.NYUDataset import NYUDataset
cost_function = nn.CrossEntropyLoss()
metric = ClassificationMetric()
# Image folder for train and val
train_loader = dl.get_image_folder_loaders(folder + "/train", NYUDataset, "SM", batch, rgb, depth)
test_loader = dl.get_image_folder_loaders(folder + "/val", NYUDataset, "NO", batch, rgb, depth)


In [12]:
MODEL = "p" # "p" or "q" or "rs" or "rp"
# Initialize the specified architecture.
if MODEL == 'p':
    model = pbnet.piggyback_net18([10,1,10], True)
elif MODEL == 'q':
    model = pbnet.quantized_net18([10,1,10], True)
elif MODEL == 'rs':
    model = rbnet.rebuffi_net18([10,1,10], serie=True, pre_imagenet=True)
elif MODEL == 'ps':
    model = rbnet.rebuffi_net18([10,1,10], serie=False, pre_imagenet=True)
else:
    raise NotImplementedError
    
model.set_index(0)

# Consider this as a sanity check
for name, par in model.named_parameters():
    if par.requires_grad:
        print(name + " requires_grad: " + str(par.requires_grad))
        

Model pretrained loaded
conv1.mask.0 requires_grad: True
bn1.0.weight requires_grad: True
bn1.0.bias requires_grad: True
layer1.0.conv1.mask.0 requires_grad: True
layer1.0.bn1.0.weight requires_grad: True
layer1.0.bn1.0.bias requires_grad: True
layer1.0.conv2.mask.0 requires_grad: True
layer1.0.bn2.0.weight requires_grad: True
layer1.0.bn2.0.bias requires_grad: True
layer1.1.conv1.mask.0 requires_grad: True
layer1.1.bn1.0.weight requires_grad: True
layer1.1.bn1.0.bias requires_grad: True
layer1.1.conv2.mask.0 requires_grad: True
layer1.1.bn2.0.weight requires_grad: True
layer1.1.bn2.0.bias requires_grad: True
layer2.0.conv1.mask.0 requires_grad: True
layer2.0.bn1.0.weight requires_grad: True
layer2.0.bn1.0.bias requires_grad: True
layer2.0.conv2.mask.0 requires_grad: True
layer2.0.bn2.0.weight requires_grad: True
layer2.0.bn2.0.bias requires_grad: True
layer2.0.downsample.0.mask.0 requires_grad: True
layer2.0.bn3.0.weight requires_grad: True
layer2.0.bn3.0.bias requires_grad: True
laye

In [26]:
from Piggyback.multiple_optim import MultipleOptimizer 

lr = 0.005
adamlr = 0.0001
decay = 0.00005
step = 10
epoch = 1
momentum = 0.9

exp = copy.deepcopy(model.state_dict())

model.bn1[2].weight.requires_grad = True # intended error to check if it works.

ignored_params = list(map(id, model.fc.parameters()))
base_params = list(filter(lambda p: (id(p) not in ignored_params) and p.requires_grad, model.parameters()))
fc_params = list(filter(lambda p: p.requires_grad, model.fc.parameters()))
# set optimizer
optimizer_a = optim.Adam(base_params, lr=adamlr, weight_decay=decay)
optimizer_b = optim.SGD(fc_params, lr=lr, momentum=momentum, weight_decay=decay)
scheduler_a = optim.lr_scheduler.StepLR(optimizer_a, step)
scheduler_b = optim.lr_scheduler.StepLR(optimizer_b, step)
scheduler = MultipleOptimizer(scheduler_a, scheduler_b)
optimizer = MultipleOptimizer(optimizer_a, optimizer_b)

## Consider this as a sanity check
#for name, par in model.named_parameters():
#    if par.requires_grad:
#        print(name + " requires_grad: " + str(par.requires_grad))

# NOTE: loss function is a parameter as well as the metric function
loss_epoch = training.train_epoch(model, 1, train_loader, optimizer, cost_function)
result = training.test_epoch(model, test_loader, cost_function, metric)

Starting time of Epoch 1: 15:36:44.973915
Train Epoch: 1 [   0/  50 ( 0%)]	AvgLoss: 1.079536
Train Epoch: 1 [  40/  50 (77%)]	AvgLoss: 2.385605


In [27]:
par = model.state_dict()
for name, par in model.named_parameters():
    if not torch.equal(exp[name], par):
        if not par.requires_grad:
            print(name + " is changed while it shouldn't")
    else:
        if par.requires_grad:
            print(name + " isn't changed while it should")

for m in model.modules():
    if isinstance(m, BasicMaskedBlock) \
    or isinstance(m, QuantizedConv2d) \
    or isinstance(m, MaskedConv2d) \
    or isinstance(m, MaskedNet) \
    or isinstance(m, BasicRebuffiBlock):
        if m.index != 0:
            print(m.__class__)

    if isinstance(m, SerieAdapterModule) \
    or isinstance(m, ParallelAdapterModule) \
    or isinstance(m, RebuffiNet):
        if m.task != 0:
            print(m.__class__)

            
# output should be:
# bn1.2.weight isn't changed while it should

bn1.2.weight isn't changed while it should


In [16]:
print(model.layer1[0].conv1.mask[0][0][0])
print(model.layer1[0].conv1.weight[0][0])
print(model.layer1[0].bn1[0].weight[0:20])
print(model.fc[2].weight[0][0:20])

tensor(1.00000e-03 *
       [[10.0000, 10.0000, 10.0000],
        [10.0000, 10.0000, 10.0000],
        [10.0000, 10.0000, 10.0000]])
tensor([[ 0.0576, -0.0951, -0.0203],
        [-0.0746, -0.7993, -0.2128],
        [ 0.0656, -0.0965, -0.0121]])
tensor([ 0.3090,  0.2147,  0.2366,  0.4259,  0.5137,  0.2181,  0.2204,
         0.2300,  0.2640,  0.2695,  0.2138,  0.4602,  0.2661,  0.2319,
         0.3900,  0.2389,  0.2660,  0.3634,  0.3474,  0.2477])
tensor(1.00000e-02 *
       [ 0.8571,  3.8492, -3.9927, -0.8347,  2.5711, -3.5917, -3.2478,
         2.8026,  0.1955, -2.8098, -0.2178, -2.3266, -2.7773,  2.0527,
        -4.3766,  1.8779, -4.0299, -0.0310,  4.0682, -0.9134])


In [23]:
print(model.layer1[0].conv1.mask[0][0][0])
print(model.layer1[0].conv1.weight[0][0])
print(model.layer1[0].bn1[0].weight[0:20])
print(model.fc[2].weight[0][0:20])

tensor(1.00000e-02 *
       [[ 0.9644,  1.0279,  1.0580],
        [ 0.9530,  0.8965,  0.9614],
        [ 1.1038,  0.8864,  0.8833]])
tensor([[ 0.0576, -0.0951, -0.0203],
        [-0.0746, -0.7993, -0.2128],
        [ 0.0656, -0.0965, -0.0121]])
tensor([ 0.3093,  0.2147,  0.2373,  0.4256,  0.5130,  0.2183,  0.2212,
         0.2305,  0.2637,  0.2704,  0.2148,  0.4612,  0.2664,  0.2314,
         0.3890,  0.2389,  0.2667,  0.3631,  0.3464,  0.2474])
tensor(1.00000e-02 *
       [ 0.8571,  3.8492, -3.9927, -0.8347,  2.5711, -3.5917, -3.2478,
         2.8026,  0.1955, -2.8098, -0.2178, -2.3266, -2.7773,  2.0527,
        -4.3766,  1.8779, -4.0299, -0.0310,  4.0682, -0.9134])
